In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from googletrans import Translator
import googletrans
%matplotlib inline

In [29]:
df_train = pd.read_csv('./data/sales_train.csv.gz',)
df_test = pd.read_csv('./data/test.csv.gz')
df_items = pd.read_csv('./data/items.csv')
df_item_categories = pd.read_pickle('./data/df_item_categories_pp.picke')
df_shop = pd.read_pickle('./data/df_shop_pp.picke')
df_sample_submission = pd.read_csv('./data/sample_submission.csv.gz')

# Train-Test 데이터 PP
### Sequece
  1. Train 데이터의 가격정보 평균값 Test에 Merge 보류 - test PP(11월 정보 반영, 아이템 평균 가격 반영)
  2. Train Test에 df_shop 정보 Merge
  3. Label Encoding  Test 데이터의 아이템 아이디 중 트레인에 없는 값이 존재함 라벨인코딩 원핫 인코딩 불가 - > 아이템 ID 대신 카테고리로 변경
  4. One-hot Encoding
  5. MinMax scaling
  6. LinearRegression

## 1. Train 데이터의 가격정보 평균값 Test에 Merge(보류)

In [30]:
df_item_cnt_mean = df_train.groupby(['shop_id','item_id']).agg({'item_cnt_day': sum})
df_item_cnt_mean.reset_index(inplace=True)
df_item_cnt_mean.head()

,shop_id,item_id,item_cnt_day
0,0,30,31.00
1,0,31,11.00
2,0,32,16.00
3,0,33,6.00
4,0,35,15.00


In [31]:
df_item_cnt_month = df_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': sum})
df_item_cnt_month.reset_index(inplace=True)
df_item_cnt_month.head()

,shop_id,item_id,date_block_num,item_cnt_day
0,0,30,1,31.00
1,0,31,1,11.00
2,0,32,0,6.00
3,0,32,1,10.00
4,0,33,0,3.00


In [32]:
df_item_cnt_month.rename(columns={'item_cnt_day' : 'item_cnt_month'},inplace=True)
df_item_cnt_month.head()

,shop_id,item_id,date_block_num,item_cnt_month
0,0,30,1,31.00
1,0,31,1,11.00
2,0,32,0,6.00
3,0,32,1,10.00
4,0,33,0,3.00


In [33]:
df_item_cnt_mean.rename(columns={'item_cnt_day' : 'item_cnt_day_mean'},inplace=True)
df_item_cnt_mean.head()

,shop_id,item_id,item_cnt_day_mean
0,0,30,31.00
1,0,31,11.00
2,0,32,16.00
3,0,33,6.00
4,0,35,15.00


In [34]:
# df_test_pp = pd.merge(df_test,df_item_cnt_month ,left_on = ['shop_id','item_id'], how='left')

In [35]:
df_train_pp = df_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_price':'mean',
                                                                            'item_cnt_day' : sum})
df_train_pp.reset_index(inplace=True)
df_train_pp.head()

,shop_id,item_id,date_block_num,item_cnt_day,item_price
0,0,30,1,31.00,265.00
1,0,31,1,11.00,434.00
2,0,32,0,6.00,221.00
3,0,32,1,10.00,221.00
4,0,33,0,3.00,347.00


#### Train

In [36]:
df_train_pp.rename(columns={'item_cnt_day':'item_cnt_month'},inplace=True)
df_train_pp.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price
0,0,30,1,31.00,265.00
1,0,31,1,11.00,434.00
2,0,32,0,6.00,221.00
3,0,32,1,10.00,221.00
4,0,33,0,3.00,347.00


In [37]:
type(df_train_pp.loc[0].date_block_num)

numpy.float64

#### Test

In [38]:
df_test.shape

(214200, 3)

In [39]:
november_column = np.zeros(shape=(214200,1),dtype=int)+11
november_column

array([[11],
       [11],
       [11],
       ..., 
       [11],
       [11],
       [11]])

In [40]:
november =  pd.DataFrame(data=november_column)


In [41]:
df_test_pp = pd.concat([df_test, november],axis=1)
df_test_pp.rename(columns={0:'date_block_num'},inplace=True)
df_test_pp.head()

,ID,shop_id,item_id,date_block_num
0,0,5,5037,11
1,1,5,5320,11
2,2,5,5233,11
3,3,5,5232,11
4,4,5,5268,11


In [42]:
df_mean_price = df_train_pp.groupby(['shop_id','item_id']).agg({'item_price' : 'mean'}).reset_index()
df_mean_price.head()

,shop_id,item_id,item_price
0,0,30,265.00
1,0,31,434.00
2,0,32,221.00
3,0,33,347.00
4,0,35,247.00


In [43]:
df_test_pp = pd.merge(df_test_pp, df_mean_price, on = ['shop_id','item_id'], how = 'left')
df_test_pp.head()

,ID,shop_id,item_id,date_block_num,item_price
0,0,5,5037,11,1693.52
1,1,5,5320,11,nan
2,2,5,5233,11,859.00
3,3,5,5232,11,599.00
4,4,5,5268,11,nan


* 5320과 같은 상품은 Train 셋에도 판매 정보가 없음  
우선 가격 칼럼을 드랍하겠음(예측 - 상품의 카테고리 정보가 핵심이 아닐까 생각해봄)

In [44]:
df_train_pp[df_train_pp['item_id']==5320]

,shop_id,item_id,date_block_num,item_cnt_month,item_price


In [45]:
df_test_pp.drop(['item_price'],axis=1, inplace=True)
df_train_pp.drop(['item_price'],axis=1, inplace=True)

## 2. Train Test에 df_shop 정보 Merge

In [46]:
df_shop.head()

,shop_name,shop_id,location_name,year2014
0,"Yakutsk Ordzhonikidze, 56 francs",0,Yakutsk,269601.00
1,"Yakutsk TC ""Central"" franc",1,Yakutsk,269601.00
2,"Adygea Shopping Center ""Mega""",2,Adygea,444403.00
3,"Balashikha TRK ""October-Kinomir""",3,Balashikha,215494.00
4,"Volzhsky shopping center ""Volga Mall""",4,Volzhsky,314255.00


#### Train

In [47]:
df_train_pp_2 = pd.merge(df_train_pp,df_shop, on='shop_id', how='left')
df_train_pp_2.head()

,shop_id,item_id,date_block_num,item_cnt_month,shop_name,location_name,year2014
0,0,30,1,31.00,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00
1,0,31,1,11.00,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00
2,0,32,0,6.00,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00
3,0,32,1,10.00,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00
4,0,33,0,3.00,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00


#### Test

In [48]:
df_test_pp_2 = pd.merge(df_test_pp,df_shop, on='shop_id', how='left')
df_test_pp_2.head()

,ID,shop_id,item_id,date_block_num,shop_name,location_name,year2014
0,0,5,5037,11,"Vologda Shopping and Entertainment Center ""Mar...",Vologda,1193371.00
1,1,5,5320,11,"Vologda Shopping and Entertainment Center ""Mar...",Vologda,1193371.00
2,2,5,5233,11,"Vologda Shopping and Entertainment Center ""Mar...",Vologda,1193371.00
3,3,5,5232,11,"Vologda Shopping and Entertainment Center ""Mar...",Vologda,1193371.00
4,4,5,5268,11,"Vologda Shopping and Entertainment Center ""Mar...",Vologda,1193371.00


In [49]:
df_train_pp_2.columns

Index(['shop_id', 'item_id', 'date_block_num', 'item_cnt_month', 'shop_name',
       'location_name', 'year2014'],
      dtype='object')

In [50]:
df_train_pp_2 = pd.DataFrame(df_train_pp_2, columns=['shop_id', 'item_id',
                                                     'date_block_num',
                                                     'shop_name','location_name', 
                                                     'year2014','item_cnt_month'])
df_train_pp_2.head()

,shop_id,item_id,date_block_num,shop_name,location_name,year2014,item_cnt_month
0,0,30,1,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00,31.00
1,0,31,1,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00,11.00
2,0,32,0,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00,6.00
3,0,32,1,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00,10.00
4,0,33,0,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,269601.00,3.00


In [51]:
df_train_pp_2.corr()

,shop_id,item_id,date_block_num,year2014,item_cnt_month
shop_id,1.00,0.02,0.02,-0.24,0.00
item_id,0.02,1.00,-0.01,0.02,-0.01
date_block_num,0.02,-0.01,1.00,-0.01,0.00
year2014,-0.24,0.02,-0.01,1.00,0.03
item_cnt_month,0.00,-0.01,0.00,0.03,1.00


In [52]:
pd.set_option('display.float_format', lambda x : '%.2f' % x) # 소수 둘째 자리까지 보이도록

In [53]:
df_test_pp_2.corr()

,ID,shop_id,item_id,date_block_num,year2014
ID,1.00,0.77,0.00,nan,-0.30
shop_id,0.77,1.00,-0.00,nan,-0.10
item_id,0.00,-0.00,1.00,nan,-0.00
date_block_num,nan,nan,nan,nan,nan
year2014,-0.30,-0.10,-0.00,nan,1.00


In [61]:
date_block_pp = {0:1,1:2,2:3,3:4,4:5,5:6,6:7,7:8,8:9,9:10,10:11,11:12,
                 12:1,13:2,14:3,15:4,16:5,17:6,18:7,19:8,20:9,21:10,22:11,23:12,
                 24:1,25:2,26:3,27:4,28:5,29:6,30:7,31:8,32:9, 33:10}

df_train_pp_2.date_block_num=df_train_pp_2.date_block_num.replace(date_block_pp)
df_train_pp_2.date_block_num.head()

0    2
1    2
2    1
3    2
4    1
Name: date_block_num, dtype: int64

## 3. Label Encoding
* Shop_id와 Name은 같은 성격을 지니기에 Name 드랍

In [62]:
from sklearn import preprocessing

In [63]:
def label_encoding(data, columns):
    label_encoder_list=[]
    for column_index in columns:
        le = preprocessing.LabelEncoder()
        le.fit(data[:,column_index])
        
        data[:,column_index]=le.transform(data[:,column_index])
        label_encoder_list.append(le)
        
        del le
        
    
    return label_encoder_list, data

#### Train makes label model

In [64]:
df_train_pp_2.drop(['shop_name'],axis=1, inplace=True)
df_test_pp_2.drop(['shop_name'],axis=1, inplace=True)

In [65]:
df_train_pp_2.head()

,shop_id,item_id,date_block_num,location_name,year2014,item_cnt_month
0,0,30,2,Yakutsk,269601.00,31.00
1,0,31,2,Yakutsk,269601.00,11.00
2,0,32,1,Yakutsk,269601.00,6.00
3,0,32,2,Yakutsk,269601.00,10.00
4,0,33,1,Yakutsk,269601.00,3.00


In [66]:
label_column = [0,1,2,3] # shop_id, item_id, date_block_num, location_name

label_encoder_list, train_data = label_encoding(df_train_pp_2.as_matrix(), label_column)

## 실험(Item_id 모델 전체 재생성)

In [67]:
df_train_pp_2.as_matrix()[:,1].shape

(1609124,)

In [68]:
item_total = pd.merge(df_train_pp_2[['item_id']],df_test_pp_2[['item_id']] ,on='item_id' ,how='outer').values

In [69]:
item_total = np.unique(item_total)

In [70]:
item_total

array([    0,     1,     2, ..., 22167, 22168, 22169], dtype=int64)

### Item _id만 별개의 인코더 생성

In [71]:
le_1 = preprocessing.LabelEncoder()
label_encoder_list[1] = le_1.fit(item_total)

In [72]:
train_data

array([[0, 30, 1, 28, 269601.0, 31.0],
       [0, 31, 1, 28, 269601.0, 11.0],
       [0, 32, 0, 28, 269601.0, 6.0],
       ..., 
       [59, 21804, 9, 29, 1271766.0, 1.0],
       [59, 21804, 11, 29, 1271766.0, 2.0],
       [59, 21804, 5, 29, 1271766.0, 1.0]], dtype=object)

In [73]:
train_data[:,1]=label_encoder_list[1].transform(train_data[:,1])

In [74]:
train_data [:10]

array([[0, 30, 1, 28, 269601.0, 31.0],
       [0, 31, 1, 28, 269601.0, 11.0],
       [0, 32, 0, 28, 269601.0, 6.0],
       [0, 32, 1, 28, 269601.0, 10.0],
       [0, 33, 0, 28, 269601.0, 3.0],
       [0, 33, 1, 28, 269601.0, 3.0],
       [0, 35, 0, 28, 269601.0, 1.0],
       [0, 35, 1, 28, 269601.0, 14.0],
       [0, 36, 1, 28, 269601.0, 1.0],
       [0, 40, 1, 28, 269601.0, 1.0]], dtype=object)

In [75]:
label_encoder_list

[LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()]

#### Model apply to Test

In [76]:
df_test_pp_2.drop(['ID'],axis=1,inplace=True)
df_test_pp_2.head()

,shop_id,item_id,date_block_num,location_name,year2014
0,5,5037,11,Vologda,1193371.00
1,5,5320,11,Vologda,1193371.00
2,5,5233,11,Vologda,1193371.00
3,5,5232,11,Vologda,1193371.00
4,5,5268,11,Vologda,1193371.00


In [77]:
test_data = df_test_pp_2.as_matrix()
test_data

array([[5, 5037, 11, 'Vologda', 1193371.0],
       [5, 5320, 11, 'Vologda', 1193371.0],
       [5, 5233, 11, 'Vologda', 1193371.0],
       ..., 
       [45, 15757, 11, 'Samara', 3211187.0],
       [45, 19648, 11, 'Samara', 3211187.0],
       [45, 969, 11, 'Samara', 3211187.0]], dtype=object)

In [78]:
test_data[:,0]=label_encoder_list[0].transform(test_data[:,0])
test_data[:,1]=label_encoder_list[1].transform(test_data[:,1])
test_data[:,2]=label_encoder_list[2].transform(test_data[:,2])
test_data[:,3]=label_encoder_list[3].transform(test_data[:,3])

test_data


#ValueError: y contains new labels: [83 140 168 173 204 237 249 254 302 402 428 430 454 480 566 639 640 762 892
#  900 907 1184 1193 1246 1252 1253 1254 1284 1386 1394 1434 1436 1437 1574
#  1575 1577 1580 1585 1679 1683 1732 1864 1867 1884 1885 2322 2323 2324 2325
#  2326 2327 2328 2420 2426 2427 2428 2434 2569 2722 2966 3022 3023 3271 3272
#  3273 3282 3284 3338 3349 3354 3405 3407 3408 3538 3571 3579 3604 3627 3760
#  3867 3908 3984 4060 4156 4310 4318 4412 4642 4925 5025 5064 5121 5248 5268
#  5269 5320 5321 5322 5441 5541 5569 5667 5721 5722 5812 5826 5843 5844 5961
#  5963 6152 6153 6169 6259 6335 6439 6519 6520 6521 6522 6523 6524 6525 6526
#  6527 6528 6529 6530 6531 6532 6533 6645 6729 6730 6731 6732 6733 6734 6742
#  6743 6863 6891 6903 6996 7094 7124 7572 7592 7607 7646 7647 7669 7727 7728
#  7729 7730 7766 7782 7862 7926 8182 8191 8337 8354 8355 8545 8549 8551 8676
#  8722 8760 8821 8822 8823 8824 8825 8826 8850 8890 8957 8968 8969 8993 9030
#  9111 9344 9530 9537 9767 9972 10028 10045 10054 10069 10176 10203 10310
#  10372 10483 10507 10747 10779 11267 11385 11762 11764 11765 11946 11974
#  12117 12409 12441 12468 12469 12470 12471 12568 12574 12591 12802 12890
#  12920 12961 13047 13096 13209 13231 13232 13242 13249 13250 13251 13257
#  13263 13264 13271 13273 13275 13284 13295 13303 13309 13310 13313 13338
#  13392 13394 13402 13445 13446 13447 13558 13559 13745 13804 13805 13846
#  14383 14521 14647 14648 14662 14739 14760 14765 14784 14832 14959 14972
#  15033 15034 15183 15329 15330 15418 15467 15553 15564 15566 15586 15665
#  15667 16051 16095 16096 16097 16099 16139 16141 16157 16193 16498 16601
#  16675 16811 16813 16976 17030 17117 17757 17858 17929 18174 18560 18627
#  18913 18979 19048 19085 19155 19210 19219 19252 19417 19651 19773 19775
#  19966 19973 20196 20378 20400 20401 20403 20439 20461 20476 20486 20535
#  20560 20561 20577 20594 20599 20600 20677 20680 20760 20796 20836 20837
#  20843 20845 20863 20878 20879 20880 20996 21196 21214 21354 21397 21453
#  21467 21708 21828 21829 21953 21974 21975 22022 22035 22137]

array([[5, 5037, 10, 25, 1193371.0],
       [5, 5320, 10, 25, 1193371.0],
       [5, 5233, 10, 25, 1193371.0],
       ..., 
       [45, 15757, 10, 19, 3211187.0],
       [45, 19648, 10, 19, 3211187.0],
       [45, 969, 10, 19, 3211187.0]], dtype=object)

## 4. One-hot Encoding

### Numeric type - > One hot encoding
*  <1609124x21932 sparse matrix of type '<class 'numpy.float64'>'
	with 6436496 stored elements in Compressed Sparse Row format>
* Memorry Error 발생


* scipy의 csr_matrix 모듈 호출 필요(Sparse한 매트릭스 처리)

### Train_data

In [79]:
def one_encoding(data, columns):
    one_encoder_list=[]
    for column_index in columns:
        one = preprocessing.OneHotEncoder()
        one.fit(data[:,column_index].reshape(-1,1))
        
        
        one_encoder_list.append(one)
        
        del one
        
    
    return one_encoder_list

In [80]:
one_encoder_list = one_encoding(train_data,[0,1,2,3])

In [81]:
one_encoder_list

[OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True)]

### Item_id one_hot 조정

In [82]:
one_hot = preprocessing.OneHotEncoder()
one_encoder_list[1] = one_hot.fit(item_total.reshape(-1,1))

In [83]:
one_encoder_list[0].transform(test_data[:,0].reshape(-1,1))

<214200x60 sparse matrix of type '<class 'numpy.float64'>'
	with 214200 stored elements in Compressed Sparse Row format>

### one hot transform

In [84]:
import scipy.sparse as sp

In [85]:
train_data_2 = one_encoder_list[0].transform(train_data[:,0].reshape(-1,1))
train_data_2 = sp.hstack((train_data_2,one_encoder_list[1].transform(train_data[:,1].reshape(-1,1))),format='csr')
train_data_2 = sp.hstack((train_data_2,one_encoder_list[2].transform(train_data[:,2].reshape(-1,1))),format='csr')
train_data_2 = sp.hstack((train_data_2,one_encoder_list[3].transform(train_data[:,3].reshape(-1,1))),format='csr')
train_data_2

<1609124x22273 sparse matrix of type '<class 'numpy.float64'>'
	with 6436496 stored elements in Compressed Sparse Row format>

In [86]:
test_data_2 = one_encoder_list[0].transform(test_data[:,0].reshape(-1,1))
test_data_2 = sp.hstack((test_data_2,one_encoder_list[1].transform(test_data[:,1].reshape(-1,1))),format='csr')
test_data_2 = sp.hstack((test_data_2,one_encoder_list[2].transform(test_data[:,2].reshape(-1,1))),format='csr')
test_data_2 = sp.hstack((test_data_2,one_encoder_list[3].transform(test_data[:,3].reshape(-1,1))),format='csr')
test_data_2

<214200x22273 sparse matrix of type '<class 'numpy.float64'>'
	with 856800 stored elements in Compressed Sparse Row format>

## 5. MinMax scaling


* csr_matrix(shop_id, item_id, date_block_num,location)와  
피쳐스케일링한 넘파이 어레이(year2014)  
hstack


* Min-Max의 값을 알 수 있기에 Min-Max 적용

In [87]:
train_data[:,4]

array([269601.0, 269601.0, 269601.0, ..., 1271766.0, 1271766.0, 1271766.0], dtype=object)

In [88]:
minmax_scale = preprocessing.MinMaxScaler().fit(train_data[:,4])
train_minmax = minmax_scale.transform(train_data[:,4])

train_minmax[:]

c:\users\cch88\miniconda3\envs\ml_python\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
c:\users\cch88\miniconda3\envs\ml_python\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
c:\users\cch88\miniconda3\envs\ml_python\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarn

array([ 0.01733807,  0.01733807,  0.01733807, ...,  0.10052229,
        0.10052229,  0.10052229])

In [89]:
test_minmax = minmax_scale.transform(test_data[:,4])

c:\users\cch88\miniconda3\envs\ml_python\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [90]:
test_minmax

array([ 0.09401515,  0.09401515,  0.09401515, ...,  0.26150299,
        0.26150299,  0.26150299])

### 스케일링한 값 csr매트릭스와 hstack ,X밸류 생성

In [91]:
train_minmax.reshape(-1,1)

array([[ 0.01733807],
       [ 0.01733807],
       [ 0.01733807],
       ..., 
       [ 0.10052229],
       [ 0.10052229],
       [ 0.10052229]])

In [92]:
x_train =sp.hstack((train_data_2,train_minmax.reshape(-1,1)),format='csr')

In [93]:
x_test = sp.hstack((test_data_2,test_minmax.reshape(-1,1)),format='csr')

## 6. LinearRegression

In [94]:
x_train

<1609124x22274 sparse matrix of type '<class 'numpy.float64'>'
	with 8001744 stored elements in Compressed Sparse Row format>

In [95]:
x_test

<214200x22274 sparse matrix of type '<class 'numpy.float64'>'
	with 1065900 stored elements in Compressed Sparse Row format>

In [96]:
y_train= train_data[:,-1].reshape(-1,1)
y_train

array([[31.0],
       [11.0],
       [6.0],
       ..., 
       [1.0],
       [2.0],
       [1.0]], dtype=object)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
slr = LinearRegression()
slr.fit(x_train,y_train) #LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [ ]:
# 모델 계수 추정
# print('Slope: %.3f' % slr.coef_)
print('===================================')
print('Intercept: %.3f' % slr.intercept_)

In [ ]:
y_test_pred = slr.predict(x_test)

In [ ]:
y_test_pred### 양수화 하기 전의 내용

In [ ]:
absolute_y_pred = np.absolute(y_test_pred)

In [ ]:
df_submit = pd.DataFrame(data=y_test_pred, columns=['item_cnt_month'])
df_submit.index.name = 'ID'
df_submit.to_csv('1st_submit_Predict_Future_Sales',sep=",")
df_submit.head()

In [ ]:
df_submit_2 = pd.DataFrame(data=absolute_y_pred, columns=['item_cnt_month'])
df_submit_2.index.name = 'ID'
df_submit_2.to_csv('Abs_trans_Predict_Future_Sales',sep=",")
df_submit_2.head()

# END RMSE 33.......
# Drop